In [1]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [2]:
agora = datetime.datetime.now(pytz.timezone('America/Sao_paulo'))
strSilverPath = f'./LAKE/SILVER/loja_simulada-vendas'
strGoldPath = f'./LAKE/GOLD/loja_simulada-vendas'

In [3]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('IngestaoGold')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )

24/06/09 12:02:21 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/06/09 12:02:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 12:02:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/09 12:02:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def pegarSilverAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strSilverPath}-{tabela}')).cache()
        print(f'Base SILVER "{tabela}" encontrada')
        return tempDF
    except Exception as e:
        print(f'Base SILVER "{tabela}" não encontrada\n', e)
        return None

In [5]:
def pegarGoldAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strGoldPath}-{tabela}')).cache()
        print(f'Base GOLD "{tabela}" encontrada')
        return tempDF
    except Exception as e:
        print(f'Base GOLD "{tabela}" não encontrada\n', e)
        return None

In [6]:
def enviarAlerta(tipoAlerta, mensagem):
    print(F'Encaminhando alerta de {tipoAlerta}')
    print(f'{tipoAlerta} - type: {type(mensagem)} - Mensagem:', mensagem)
    return

In [7]:
def salvarDadosDoDF(df,pathFile):
    return df.write.mode("overwrite").parquet(pathFile)

In [8]:
# DIMENSAO TEMPO
def tempo():
    tabela = 'dimensao_tempo'
    goldDF = pegarGoldAtual(tabela)
    if goldDF != None:
        enviarAlerta('Informacao', f'{tabela} MAX DATA: "{goldDF.select(F.max("data").alias("MAX_DATA")).collect()[0]["MAX_DATA"]}"')
        return
    refDF = spark.createDataFrame([(1,)], ['id', ])
    windowSpec = Window.orderBy("data")
    tempoDF = (refDF
        # .withColumn('data', F.explode(F.expr("sequence(to_date('2024-06-01'), to_date('2024-06-10'), interval 1 day)")))
        .withColumn('data', F.explode(F.expr("sequence(to_date('2010-01-01'), to_date('2030-12-31'), interval 1 day)")))
        .withColumn('dia', F.day(F.col('data')))
        .withColumn('mes', F.month(F.col('data')))
        .withColumn('ano', F.year(F.col('data')))
        .withColumn('dia_semana', F.weekday(F.col('data')))  # 0 - 6 => segunda - domingo
        .withColumn('trimestre', F.quarter(F.col('data')))
        .withColumn("chave_tempo", F.row_number().over(windowSpec))
        .drop('id')
    )
    salvarDadosDoDF(tempoDF, f'{strGoldPath}-{tabela}')
    enviarAlerta('Informacao',
        f'{tabela} criada com MAX DATA: "{tempoDF.select(F.max("data").alias("MAX_DATA")).collect()[0]["MAX_DATA"]}"')
    return

In [9]:
# DIMENSAO CLIENTE
def cliente():
    tabela = 'dimensao_cliente'
    silverDF = pegarSilverAtual('clientes')
    if silverDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "{tabela}" interrompida, base SILVER não encontrada')
        return
    goldDF = (silverDF
        .select(F.col('surrogate_key').alias('chave_cliente'),'id_cliente','cliente','estado','origem_racial','sexo','status')
        .withColumn("data_carga", F.lit(agora.strftime("%Y-%m-%d")).cast('date'))
     )
    salvarDadosDoDF(goldDF, f'{strGoldPath}-{tabela}')
    enviarAlerta('Informacao',
        f'Registros a serem salvos em "{tabela}": {goldDF.select(F.count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [10]:
# DIMENSAO VENDEDOR
def vendedor():
    tabela = 'dimensao_vendedor'
    silverDF = pegarSilverAtual('vendedores')
    if silverDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "{tabela}" interrompida, base SILVER não encontrada')
        return
    goldDF = (silverDF
        .select(F.col('surrogate_key').alias('chave_vendedor'),'id_vendedor','nome')
        .withColumn("data_carga", F.lit(agora.strftime("%Y-%m-%d")).cast('date'))
     )
    salvarDadosDoDF(goldDF, f'{strGoldPath}-{tabela}')
    enviarAlerta('Informacao',
        f'Registros a serem salvos em "{tabela}": {goldDF.select(F.count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [11]:
# DIMENSAO PRODUTO
def produto():
    tabela = 'dimensao_produto'
    silverDF = pegarSilverAtual('produtos')
    if silverDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "{tabela}" interrompida, base SILVER não encontrada')
        return
    goldDF = (silverDF
        .select(F.col('surrogate_key').alias('chave_produto'),'id_produto','produto'
            ,F.col('data_carga').alias('data_ref'))
        .withColumn("data_carga", F.lit(agora.strftime("%Y-%m-%d")).cast('date'))
     )
    salvarDadosDoDF(goldDF, f'{strGoldPath}-{tabela}')
    enviarAlerta('Informacao',
        f'Registros a serem salvos em "{tabela}": {goldDF.select(F.count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [12]:
# FATO VENDA
def venda():
    tabela = 'fato_venda'
    silverVendasDF = pegarSilverAtual('vendas')
    silverItens_vendasDF = pegarSilverAtual('itens_venda')
    goldClientesDF = pegarGoldAtual('dimensao_cliente')
    goldVendedoresDF = pegarGoldAtual('dimensao_vendedor')
    goldProdutosDF = pegarGoldAtual('dimensao_produto')
    goldTempoDF = pegarGoldAtual('dimensao_tempo')
    if silverVendasDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "vendas" interrompida, base SILVER não encontrada')
        return
    if silverItens_vendasDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "itens_vendas" interrompida, base SILVER não encontrada')
        return
    if goldClientesDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "dimensao_cliente" interrompida, base SILVER não encontrada')
        return
    if goldVendedoresDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "dimensao_vendedor" interrompida, base SILVER não encontrada')
        return
    if goldProdutosDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "dimensao_produto" interrompida, base SILVER não encontrada')
        return
    if goldTempoDF == None:
        enviarAlerta('Informacao', f'Ingestão SILVER - GOLD base "tempo" interrompida, base SILVER não encontrada')
        return
    goldBaseDF = (silverVendasDF
        .join(silverItens_vendasDF, silverVendasDF.id_venda == silverItens_vendasDF.id_venda, 'fullouter')
        .select(silverVendasDF.id_venda, silverVendasDF.id_vendedor,silverVendasDF.id_cliente, silverVendasDF.data_venda
                ,silverItens_vendasDF.id_produto,silverItens_vendasDF.quantidade,silverItens_vendasDF.valor_unitario
                ,silverItens_vendasDF.valor_total,silverItens_vendasDF.desconto)
         )
    gold1DF = (goldBaseDF
        .join(goldClientesDF,((goldBaseDF.id_cliente == goldClientesDF.id_cliente)), 'inner')
        .select(goldBaseDF.id_venda,goldBaseDF.id_vendedor,goldBaseDF.data_venda,goldBaseDF.id_produto
            ,goldClientesDF.chave_cliente,goldBaseDF.quantidade,goldBaseDF.valor_unitario,goldBaseDF.valor_total
            ,goldBaseDF.desconto)
         )
    gold2DF = (gold1DF
        .join(goldProdutosDF,((gold1DF.id_produto == goldProdutosDF.id_produto)), 'inner')
        .select(gold1DF.id_venda, gold1DF.id_vendedor, gold1DF.data_venda, goldProdutosDF.chave_produto
            ,gold1DF.chave_cliente, gold1DF.quantidade, gold1DF.valor_unitario, gold1DF.valor_total
            ,gold1DF.desconto, goldProdutosDF.id_produto)
        )
    gold3DF = (gold2DF
        .join(goldVendedoresDF,((gold2DF.id_vendedor == goldVendedoresDF.id_vendedor)), 'inner')
        .select(gold2DF.id_venda,gold2DF.chave_produto,goldVendedoresDF.chave_vendedor,gold2DF.chave_cliente
            ,gold2DF.data_venda,gold2DF.quantidade,gold2DF.valor_unitario,gold2DF.valor_total
            ,gold2DF.desconto)
        )
    gold4DF = (gold3DF
        .join(goldTempoDF,gold3DF.data_venda == goldTempoDF.data, 'inner')
        .select(gold3DF.id_venda,gold3DF.chave_produto,goldVendedoresDF.chave_vendedor,gold3DF.chave_cliente
            ,goldTempoDF.chave_tempo,gold3DF.quantidade,gold3DF.valor_unitario,gold3DF.valor_total
            ,gold3DF.desconto)
        )
    windowSpec = Window.orderBy("id_venda")        
    gold5DF = (gold4DF.withColumn("chave_vendas", F.row_number().over(windowSpec))
                .drop('id_venda')
                .withColumn("data_carga", F.lit(agora.strftime("%Y-%m-%d")).cast('date'))
              )
    salvarDadosDoDF(gold5DF, f'{strGoldPath}-{tabela}')
    enviarAlerta('Informacao',
        f'Registros a serem salvos em "{tabela}": {gold5DF.select(F.count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

venda()

Base SILVER "vendas" encontrada
Base SILVER "itens_venda" encontrada
Base GOLD "dimensao_cliente" encontrada
Base GOLD "dimensao_vendedor" encontrada
Base GOLD "dimensao_produto" encontrada
Base GOLD "dimensao_tempo" encontrada


24/06/09 12:02:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 1

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos em "fato_venda": 72633


In [13]:
def executar():
    tempo()
    cliente()
    vendedor()
    produto()
    venda()
    return

In [14]:
def main():
    try:
        enviarAlerta('Informacao', 'Ingestão para base GOLD iniciada...')
        executar()
        enviarAlerta('Informacao', 'Ingestão para base GOLD concluída com SUCESSO!!!')
    except Exception as e:
        enviarAlerta('Erro', e)
    finally:
        spark.stop()
        print('conexão spark finalizada!')

In [15]:
main()

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Ingestão para base GOLD iniciada...
Base GOLD "dimensao_tempo" encontrada


24/06/09 12:02:32 WARN CacheManager: Asked to cache already cached data.


Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: dimensao_tempo MAX DATA: "2030-12-31"
Base SILVER "clientes" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos em "dimensao_cliente": 530
Base SILVER "vendedores" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos em "dimensao_vendedor": 36
Base SILVER "produtos" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos em "dimensao_produto": 14
Base SILVER "vendas" encontrada
Base SILVER "itens_venda" encontrada
Base GOLD "dimensao_cliente" encontrada


24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.
24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.
24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.
24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.
24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.
24/06/09 12:02:34 WARN CacheManager: Asked to cache already cached data.


Base GOLD "dimensao_vendedor" encontrada
Base GOLD "dimensao_produto" encontrada
Base GOLD "dimensao_tempo" encontrada


24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 12:02:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 1

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos em "fato_venda": 72633
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Ingestão para base GOLD concluída com SUCESSO!!!
conexão spark finalizada!


***